In [8]:
# import packages
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, cross_val_score
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report, roc_curve, roc_auc_score
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import StratifiedKFold

from utils import feature_engineering, additional_feature_engineering
# set the aesthetic style of the plots
sns.set_style()

# filter warning messages
import warnings
warnings.filterwarnings('ignore')

In [9]:
df_credit = pd.read_csv('saved/preprocessed_bank_data.csv')
df_credit.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41741 entries, 0 to 41740
Data columns (total 27 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   target_default                                   41741 non-null  bool   
 1   score_1                                          41741 non-null  object 
 2   score_2                                          41741 non-null  object 
 3   score_3                                          41741 non-null  float64
 4   score_4                                          41741 non-null  float64
 5   score_5                                          41741 non-null  float64
 6   score_6                                          41741 non-null  float64
 7   risk_rate                                        41741 non-null  float64
 8   last_amount_borrowed                             41741 non-null  float64
 9   last_borrowed_in_months     

In [10]:
X_processed = feature_engineering(df_credit)
X_processed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41741 entries, 0 to 41740
Data columns (total 42 columns):
 #   Column                                           Non-Null Count  Dtype   
---  ------                                           --------------  -----   
 0   target_default                                   41741 non-null  bool    
 1   score_1                                          41741 non-null  object  
 2   score_2                                          41741 non-null  object  
 3   score_3                                          41741 non-null  float64 
 4   score_4                                          41741 non-null  float64 
 5   score_5                                          41741 non-null  float64 
 6   score_6                                          41741 non-null  float64 
 7   risk_rate                                        41741 non-null  float64 
 8   last_amount_borrowed                             41741 non-null  float64 
 9   last_borrowed_in_

In [11]:
import pickle
categorical_cols = X_processed.select_dtypes(exclude=['float64', 'int64']).columns.tolist()

label_encoders = {}
for col in categorical_cols:
    label_encoders[col] = LabelEncoder()
    X_processed[col] = label_encoders[col].fit_transform(X_processed[col])




# Save label encoders to disk
with open('saved/label_encoders.pkl', 'wb') as f:
    pickle.dump(label_encoders, f)

In [12]:
X_processed2 = additional_feature_engineering(X_processed)
X_processed2

,target_default,score_1,score_2,score_3,score_4,score_5,score_6,risk_rate,last_amount_borrowed,last_borrowed_in_months,...,score_1_bin_x_score_2_bin,score_1_bin_x_fraud_score_bin,score_2_bin_x_fraud_score_bin,state_x_real_state,state_x_shipping_state,state_real_state_avg_score_1,state_shipping_state_avg_score_2,score_1_sq,score_2_sq,facebook_profile_sq
0,0,0,10,350.0,101.800832,0.259555,108.427273,0.40,25033.92,36.0,...,0,0,0,44_1,44_12,1.897115,17.494505,0,100,1
1,0,3,16,370.0,97.062615,0.942655,92.002546,0.24,0.00,0.0,...,0,0,0,47_1,47_20,1.859049,17.294872,9,256,0
2,1,3,9,360.0,100.027073,0.351918,112.892453,0.29,7207.92,36.0,...,0,0,0,32_3,32_19,1.922201,16.658537,9,81,0
3,0,0,21,510.0,101.599485,0.987673,94.902491,0.32,0.00,0.0,...,0,0,0,23_3,23_17,1.915888,17.650000,0,441,0
4,0,2,1,500.0,98.474289,0.532539,118.126207,0.18,0.00,0.0,...,0,0,0,47_3,47_12,1.947590,17.600000,4,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41736,0,3,16,280.0,96.379531,0.416693,103.667082,0.17,14766.42,36.0,...,0,0,0,46_1,46_8,1.804416,18.303030,9,256,0
41737,0,6,31,370.0,96.124977,0.692196,97.977973,0.27,0.00,0.0,...,0,0,0,36_1,36_18,2.000000,23.000000,36,961,0
41738,0,4,24,280.0,102.377780,0.530938,93.687747,0.30,0.00,0.0,...,0,0,0,47_4,47_23,1.922551,17.456029,16,576,0
41739,1,6,5,240.0,100.476090,0.214697,86.759074,0.37,0.00,0.0,...,0,0,0,47_1,47_23,1.859049,17.456029,36,25,1


In [13]:
df_credit.to_csv('saved/feature_engineered_data.csv', index = False)